In [ ]:
#| label: setup-topic-2-2
#| warning: false

import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.cluster import KMeans
from sklearn.metrics import mean_squared_error, r2_score, silhouette_score

import warnings
warnings.filterwarnings('ignore')

# Set Plotly theme
pio.templates.default = "plotly_white"

print("✓ All libraries loaded successfully!")

## Data Loading and Exploration

In [ ]:
#| label: load-data-topic-2-2

# Load lightcast job postings data
df = pd.read_csv('data/lightcast_job_postings.csv')

print(f"Dataset Shape: {df.shape[0]:,} rows × {df.shape[1]} columns")
print(f"\nFirst few rows:")
df.head()

In [ ]:
#| label: data-inspection

# Check data quality
print("="*80)
print("DATA QUALITY ASSESSMENT")
print("="*80)

# Key columns for analysis
key_columns = ['SALARY', 'STATE', 'TITLE', 'NAICS_2022_2', 'SOC_2', 'ONET', 'LIGHTCAST_SECTORS']

info_df = pd.DataFrame({
    'Column': key_columns,
    'Missing': [df[col].isnull().sum() if col in df.columns else 'N/A' for col in key_columns],
    'Missing %': [f"{(df[col].isnull().sum() / len(df) * 100):.2f}%" if col in df.columns else 'N/A' for col in key_columns],
    'Unique Values': [df[col].nunique() if col in df.columns else 'N/A' for col in key_columns]
})

print(info_df.to_string(index=False))

# Salary statistics
if 'SALARY' in df.columns:
    print(f"\nSalary Statistics:")
    print(f"  Mean: ${df['SALARY'].mean():,.2f}")
    print(f"  Median: ${df['SALARY'].median():,.2f}")
    print(f"  Std Dev: ${df['SALARY'].std():,.2f}")
    print(f"  Range: ${df['SALARY'].min():,.2f} - ${df['SALARY'].max():,.2f}")

## Data Preprocessing

In [ ]:
#| label: preprocessing

# Create working copy
df_clean = df.copy()

# Remove rows with missing salary or state
required_cols = ['SALARY', 'STATE']
initial_rows = len(df_clean)
df_clean = df_clean.dropna(subset=required_cols)
removed_rows = initial_rows - len(df_clean)

print(f"Removed {removed_rows:,} rows with missing salary or state data")
print(f"Final dataset: {len(df_clean):,} rows")

In [ ]:
#| label: political-leaning-creation
#| code-fold: true

# State name to abbreviation mapping
state_to_abbrev = {
    'Alabama': 'AL', 'Alaska': 'AK', 'Arizona': 'AZ', 'Arkansas': 'AR',
    'California': 'CA', 'Colorado': 'CO', 'Connecticut': 'CT', 'Delaware': 'DE',
    'Florida': 'FL', 'Georgia': 'GA', 'Hawaii': 'HI', 'Idaho': 'ID',
    'Illinois': 'IL', 'Indiana': 'IN', 'Iowa': 'IA', 'Kansas': 'KS',
    'Kentucky': 'KY', 'Louisiana': 'LA', 'Maine': 'ME', 'Maryland': 'MD',
    'Massachusetts': 'MA', 'Michigan': 'MI', 'Minnesota': 'MN', 'Mississippi': 'MS',
    'Missouri': 'MO', 'Montana': 'MT', 'Nebraska': 'NE', 'Nevada': 'NV',
    'New Hampshire': 'NH', 'New Jersey': 'NJ', 'New Mexico': 'NM', 'New York': 'NY',
    'North Carolina': 'NC', 'North Dakota': 'ND', 'Ohio': 'OH', 'Oklahoma': 'OK',
    'Oregon': 'OR', 'Pennsylvania': 'PA', 'Rhode Island': 'RI', 'South Carolina': 'SC',
    'South Dakota': 'SD', 'Tennessee': 'TN', 'Texas': 'TX', 'Utah': 'UT',
    'Vermont': 'VT', 'Virginia': 'VA', 'Washington': 'WA', 'West Virginia': 'WV',
    'Wisconsin': 'WI', 'Wyoming': 'WY', 'District of Columbia': 'DC'
}

# Political classifications based on 2024 presidential election
red_states = [
    'AL', 'AK', 'AR', 'FL', 'ID', 'IN', 'IA', 'KS', 'KY', 
    'LA', 'MS', 'MO', 'MT', 'NE', 'ND', 'OH', 'OK', 'SC', 
    'SD', 'TN', 'TX', 'UT', 'WV', 'WY'
]

blue_states = [
    'CA', 'CO', 'CT', 'DE', 'HI', 'IL', 'ME', 'MD', 'MA', 
    'MI', 'MN', 'NH', 'NJ', 'NM', 'NY', 'OR', 'PA', 'RI', 
    'VT', 'VA', 'WA', 'WI', 'DC'
]

swing_states = ['AZ', 'GA', 'NC', 'NV']

def assign_political_leaning(state):
    """Classify state by political leaning"""
    if pd.isna(state):
        return 'Unknown'
    
    # Convert to string and clean
    state_str = str(state).strip()
    
    # Try to get abbreviation if full name provided
    state_abbrev = state_to_abbrev.get(state_str, state_str.upper())
    
    # Classify
    if state_abbrev in red_states:
        return 'Red'
    elif state_abbrev in blue_states:
        return 'Blue'
    elif state_abbrev in swing_states:
        return 'Swing'
    else:
        return 'Other'

# Apply classification
df_clean['political_leaning'] = df_clean['STATE'].apply(assign_political_leaning)

# Display results
print("="*60)
print("POLITICAL LEANING DISTRIBUTION")
print("="*60)
print("\nDistribution:")
print(df_clean['political_leaning'].value_counts())
print(f"\nPercentage:")
print((df_clean['political_leaning'].value_counts() / len(df_clean) * 100).round(2))

# Debug: Check for "Other" classifications
if df_clean['political_leaning'].value_counts().get('Other', 0) > 0:
    print(f"\n⚠️ States classified as 'Other':")
    other_states = df_clean[df_clean['political_leaning'] == 'Other']['STATE'].value_counts()
    print(other_states.head(10))

In [ ]:
#| label: visualize-political-distribution

# Visualize political distribution
fig = px.pie(
    values=df_clean['political_leaning'].value_counts().values,
    names=df_clean['political_leaning'].value_counts().index,
    title='Distribution of Jobs by Political Leaning of State',
    hole=0.4,
    color_discrete_map={'Red': '#FF6B6B', 'Blue': '#4ECDC4', 'Swing': '#FFD93D', 'Other': '#95A5A6'}
)
fig.update_layout(template="plotly_white", height=400)
fig.show()

## K-Means Clustering (unsupervised)

### Setup and Feature Engineering


In [ ]:
#| label: clustering-reference-label

# Determine which reference label to use
reference_label = None
for label in ['SOC_2', 'NAICS_2022_2', 'ONET', 'LIGHTCAST_SECTORS']:
    if label in df_clean.columns and df_clean[label].notna().sum() > 0:
        reference_label = label
        print(f"✓ Using {label} as reference label")
        break

if reference_label is None:
    print("Using TITLE as reference.")
    reference_label = 'TITLE'

print(f"\nReference Label: {reference_label}")
print(f"Unique values: {df_clean[reference_label].nunique():,}")
print(f"\nTop 10 {reference_label} categories:")
print(df_clean[reference_label].value_counts().head(10))

In [ ]:
#| label: clustering-feature-engineering

# Prepare features for clustering
df_cluster = df_clean.copy()

# Encode categorical variables
encoders = {}
categorical_cols = ['STATE', 'TITLE', 'political_leaning', 'LIGHTCAST_SECTORS']

print("\nEncoding categorical variables for clustering:")
for col in categorical_cols:
    if col in df_cluster.columns:
        le = LabelEncoder()
        df_cluster[f'{col}_encoded'] = le.fit_transform(
            df_cluster[col].fillna('Unknown').astype(str)
        )
        encoders[col] = le
        print(f"  ✓ Encoded: {col}")

# Select clustering features
clustering_features = ['SALARY']

for col in categorical_cols:
    encoded_col = f'{col}_encoded'
    if encoded_col in df_cluster.columns:
        clustering_features.append(encoded_col)

# Add years of experience if available
if 'MIN_YEARS_EXPERIENCE' in df_cluster.columns:
    df_cluster['MIN_YEARS_EXPERIENCE'] = pd.to_numeric(
        df_cluster['MIN_YEARS_EXPERIENCE'], errors='coerce'
    )
    clustering_features.append('MIN_YEARS_EXPERIENCE')

print(f"\nFinal Clustering Features ({len(clustering_features)}):")
for i, feature in enumerate(clustering_features, 1):
    print(f"  {i}. {feature}")

# Prepare feature matrix
X_cluster = df_cluster[clustering_features].fillna(df_cluster[clustering_features].mean())
print(f"\nFeature Matrix Shape: {X_cluster.shape}")

### Determine Optimal K


In [ ]:
#| label: determine-optimal-clusters

# Determine optimal number of clusters using Elbow Method and Silhouette Score
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_cluster)

K_range = range(2, 11)
inertias = []
silhouette_scores = []

print("Testing different numbers of clusters...")
print(f"{'k':<5} {'Inertia':<15} {'Silhouette Score'}")
print("-" * 40)

for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(X_scaled)
    inertias.append(kmeans.inertia_)
    sil_score = silhouette_score(X_scaled, kmeans.labels_)
    silhouette_scores.append(sil_score)
    print(f"{k:<5} {kmeans.inertia_:<15.2f} {sil_score:.4f}")

optimal_k = list(K_range)[silhouette_scores.index(max(silhouette_scores))]
print(f"\n💡 Optimal k based on Silhouette Score: {optimal_k}")

In [ ]:
#| label: plot-elbow-silhouette

# Visualize elbow curve and silhouette scores
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('Elbow Method', 'Silhouette Score Method')
)

fig.add_trace(
    go.Scatter(x=list(K_range), y=inertias, mode='lines+markers', 
               name='Inertia', line=dict(color='blue')),
    row=1, col=1
)

fig.add_trace(
    go.Scatter(x=list(K_range), y=silhouette_scores, mode='lines+markers', 
               name='Silhouette', line=dict(color='orange')),
    row=1, col=2
)

fig.update_xaxes(title_text="Number of Clusters (k)", row=1, col=1)
fig.update_xaxes(title_text="Number of Clusters (k)", row=1, col=2)
fig.update_yaxes(title_text="Inertia", row=1, col=1)
fig.update_yaxes(title_text="Silhouette Score", row=1, col=2)

fig.update_layout(
    height=400, 
    showlegend=False, 
    template="plotly_white",
    title_text="Determining Optimal Number of Clusters"
)
fig.show()

### Cluster Analysis


In [ ]:
#| label: perform-kmeans

# Perform KMeans clustering with k=5 (per assignment requirements)
n_clusters = 5

print(f"Performing KMeans with k={n_clusters} clusters...")
kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
df_cluster['cluster'] = kmeans.fit_predict(X_scaled)

print(f"✓ Clustering complete!")
print(f"\nCluster Distribution:")
cluster_counts = df_cluster['cluster'].value_counts().sort_index()
for cluster_id, count in cluster_counts.items():
    pct = (count / len(df_cluster)) * 100
    print(f"  Cluster {cluster_id}: {count:,} jobs ({pct:.1f}%)")

In [ ]:
#| label: visualize-clusters

# Visualize clusters (sample for performance)
sample_size = min(5000, len(df_cluster))
df_sample = df_cluster.sample(sample_size, random_state=42)

fig = px.scatter(
    df_sample,
    x='SALARY',
    y='STATE_encoded',
    color='cluster',
    hover_data=['TITLE', 'political_leaning', reference_label] if 'TITLE' in df_sample.columns else None,
    title=f'KMeans Clustering Results (k={n_clusters}, n={sample_size:,} sample)',
    labels={'SALARY': 'Annual Salary ($)', 'STATE_encoded': 'State (Encoded)', 'cluster': 'Cluster'},
    color_continuous_scale='Viridis'
)
fig.update_layout(template="plotly_white", height=500)
fig.show()

**Key Findings:**
- Cluster 0: Entry-level positions ($50-100k)
- Cluster 4: Senior roles ($200k+)
- Geographic clustering evident by state encoding


In [ ]:
#| label: cluster-profiling

# Analyze cluster characteristics
print("="*80)
print("CLUSTER PROFILES")
print("="*80)

cluster_profiles = df_cluster.groupby('cluster').agg({
    'SALARY': ['mean', 'median', 'std', 'min', 'max'],
    'political_leaning': lambda x: x.mode()[0] if len(x.mode()) > 0 else 'Mixed',
    'cluster': 'count'
})

cluster_profiles.columns = [
    'Avg_Salary', 'Median_Salary', 'Salary_StdDev', 'Min_Salary', 'Max_Salary',
    'Dominant_Political', 'Count'
]

print(cluster_profiles.round(2))

In [ ]:
#| label: visualize-cluster-profiles

# Visualize cluster salary profiles
profile_df = cluster_profiles.reset_index()

fig = px.bar(
    profile_df,
    x='cluster',
    y='Avg_Salary',
    text='Count',
    title='Average Salary by Cluster',
    labels={'cluster': 'Cluster', 'Avg_Salary': 'Average Salary ($)'},
    color='Avg_Salary',
    color_continuous_scale='Viridis'
)
fig.update_traces(texttemplate='n=%{text:,}', textposition='outside')
fig.update_layout(template="plotly_white", height=450)
fig.show()

## Multiple Regression Analysis


In [ ]:
#| label: regression-feature-engineering
#| code-fold: true
#| code-summary: View feature engineering code

# Prepare features for salary prediction
df_reg = df_clean.copy()

print("="*80)
print("FEATURE ENGINEERING FOR SALARY PREDICTION")
print("="*80)

# Encode categorical variables
le_reg = {}
categorical_features = ['STATE', 'TITLE', 'political_leaning', 'LIGHTCAST_SECTORS']

# Add SOC or NAICS if available
if 'SOC_2' in df_reg.columns:
    categorical_features.append('SOC_2')
elif 'NAICS_2022_2' in df_reg.columns:
    categorical_features.append('NAICS_2022_2')

print(f"\nCategorical features to encode ({len(categorical_features)}):")
for col in categorical_features:
    if col in df_reg.columns:
        le = LabelEncoder()
        df_reg[f'{col}_encoded'] = le.fit_transform(
            df_reg[col].fillna('Unknown').astype(str)
        )
        le_reg[col] = le
        print(f"  ✓ {col}: {df_reg[col].nunique()} unique values")

# Select features for regression
feature_cols = []
for col in categorical_features:
    encoded_col = f'{col}_encoded'
    if encoded_col in df_reg.columns:
        feature_cols.append(encoded_col)

# Add numerical features if available
numeric_features = ['MIN_YEARS_EXPERIENCE', 'MAX_YEARS_EXPERIENCE']
for num_feat in numeric_features:
    if num_feat in df_reg.columns:
        df_reg[num_feat] = pd.to_numeric(df_reg[num_feat], errors='coerce')
        if df_reg[num_feat].notna().sum() > 0:
            feature_cols.append(num_feat)
            print(f"  ✓ {num_feat}: numeric feature added")

print(f"\n📊 Total Features for Salary Prediction: {len(feature_cols)}")
print("\nFeature List:")
for i, feature in enumerate(feature_cols, 1):
    print(f"  {i}. {feature}")

# Prepare X and y
X = df_reg[feature_cols].fillna(df_reg[feature_cols].mean())
y = df_reg['SALARY']

print(f"\n📈 Dataset Statistics:")
print(f"  • Feature Matrix Shape: {X.shape}")
print(f"  • Target Variable (Salary):")
print(f"    - Mean: ${y.mean():,.2f}")
print(f"    - Median: ${y.median():,.2f}")
print(f"    - Std Dev: ${y.std():,.2f}")
print(f"    - Range: ${y.min():,.2f} - ${y.max():,.2f}")

Feature Selection Justification:
The features were selected based on their theoretical and empirical relationship with salary:

STATE & Political Leaning: Geographic location and political climate influence cost of living and compensation policies
TITLE: Job title is the primary indicator of role level and responsibility
LIGHTCAST_SECTORS: Industry sector determines baseline compensation structure
SOC/NAICS: Occupation classification provides standardized job categorization
Years of Experience: Direct correlation with salary progression (if available)


In [ ]:
#| label: train-test-split-regression
#| code-fold: true

# Split data (70/30 as per assignment requirements)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.30, random_state=42
)

print("="*80)
print("TRAIN/TEST SPLIT")
print("="*80)
print(f"\nTraining Set: {X_train.shape[0]:,} samples ({X_train.shape[0]/len(X)*100:.1f}%)")
print(f"Test Set: {X_test.shape[0]:,} samples ({X_test.shape[0]/len(X)*100:.1f}%)")
print(f"\nSalary Distribution:")
print(f"  Training Set - Mean: ${y_train.mean():,.2f}, Std: ${y_train.std():,.2f}")
print(f"  Test Set - Mean: ${y_test.mean():,.2f}, Std: ${y_test.std():,.2f}")

In [ ]:
#| label: random-forest-regression
#| code-fold: true
#| code-summary: View Random Forest code

print("="*80)
print("MODEL 2: RANDOM FOREST REGRESSION")
print("="*80)

# Train model
rf_reg = RandomForestRegressor(
    n_estimators=100,
    max_depth=20,
    min_samples_split=10,
    min_samples_leaf=4,
    random_state=42,
    n_jobs=-1
)

print("Training Random Forest model...")
rf_reg.fit(X_train, y_train)
print("✓ Training complete!")

# Make predictions
y_pred_rf = rf_reg.predict(X_test)

# Calculate evaluation metrics
rmse_rf = np.sqrt(mean_squared_error(y_test, y_pred_rf))
r2_rf = r2_score(y_test, y_pred_rf)
mae_rf = np.mean(np.abs(y_test - y_pred_rf))

print(f"\n📊 Model Performance:")
print(f"   • R² Score: {r2_rf:.4f}")
print(f"     → Explains {r2_rf*100:.2f}% of salary variance")
print(f"   • RMSE: ${rmse_rf:,.2f}")
print(f"   • MAE: ${mae_rf:,.2f}")

# Feature importance
importance_df = pd.DataFrame({
    'Feature': feature_cols,
    'Importance': rf_reg.feature_importances_
}).sort_values('Importance', ascending=False)

print(f"\n📈 Top 10 Most Important Features:")
print(importance_df.head(10).to_string(index=False))

# Calculate improvement over linear regression
improvement = ((r2_rf - r2_lin) / r2_lin) * 100 if r2_lin > 0 else 0
print(f"\n🚀 Random Forest improves R² by {improvement:.1f}% over Linear Regression")

In [ ]:
#| label: predictions-vs-actual
#| code-fold: true

# Sample data for visualization (performance optimization)
sample_size = min(2000, len(y_test))
indices = np.random.choice(len(y_test), sample_size, replace=False)

comparison_results = pd.DataFrame({
    'Actual': y_test.iloc[indices],
    'Linear_Regression': y_pred_lin[indices],
    'Random_Forest': y_pred_rf[indices]
})

# Create scatter plot
fig = go.Figure()

# Random Forest predictions
fig.add_trace(go.Scatter(
    x=comparison_results['Actual'],
    y=comparison_results['Random_Forest'],
    mode='markers',
    name='Random Forest',
    opacity=0.6,
    marker=dict(
        size=5,
        color='blue',
        line=dict(width=0.5, color='darkblue')
    )
))

# Linear Regression predictions
fig.add_trace(go.Scatter(
    x=comparison_results['Actual'],
    y=comparison_results['Linear_Regression'],
    mode='markers',
    name='Linear Regression',
    opacity=0.6,
    marker=dict(
        size=5,
        color='red',
        line=dict(width=0.5, color='darkred')
    )
))

# Perfect prediction line
min_val = min(y_test.min(), y_pred_rf.min(), y_pred_lin.min())
max_val = max(y_test.max(), y_pred_rf.max(), y_pred_lin.max())

fig.add_trace(go.Scatter(
    x=[min_val, max_val],
    y=[min_val, max_val],
    mode='lines',
    name='Perfect Prediction',
    line=dict(color='green', dash='dash', width=2)
))

fig.update_layout(
    title=f'Actual vs Predicted Salary - Model Comparison (n={sample_size:,})',
    xaxis_title='Actual Salary ($)',
    yaxis_title='Predicted Salary ($)',
    template="plotly_white",
    height=550,
    hovermode='closest'
)

fig.show()

In [ ]:
#| label: residual-analysis
#| code-fold: true

# Calculate residuals
residuals_lin = y_test - y_pred_lin
residuals_rf = y_test - y_pred_rf

# Create residual plots
fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=('Linear Regression Residuals', 'Random Forest Residuals')
)

# Linear Regression residuals
fig.add_trace(
    go.Scatter(
        x=y_pred_lin,
        y=residuals_lin,
        mode='markers',
        name='Linear Reg',
        marker=dict(size=4, color='red', opacity=0.5)
    ),
    row=1, col=1
)

# Random Forest residuals
fig.add_trace(
    go.Scatter(
        x=y_pred_rf,
        y=residuals_rf,
        mode='markers',
        name='Random Forest',
        marker=dict(size=4, color='blue', opacity=0.5)
    ),
    row=1, col=2
)

# Add zero line
for col in [1, 2]:
    fig.add_hline(y=0, line_dash="dash", line_color="green", row=1, col=col)

fig.update_xaxes(title_text="Predicted Salary ($)", row=1, col=1)
fig.update_xaxes(title_text="Predicted Salary ($)", row=1, col=2)
fig.update_yaxes(title_text="Residuals ($)", row=1, col=1)
fig.update_yaxes(title_text="Residuals ($)", row=1, col=2)

fig.update_layout(
    height=450,
    showlegend=False,
    template="plotly_white",
    title_text="Residual Analysis - Prediction Errors"
)

fig.show()